In [1]:
import pandas as pd
import P3_04_module_fao as p3

In [2]:
#CREATION DU CSV POUR LA TABLE POPULATION

# usecols : charger les données utiles
population = pd.read_csv('./DATA/fr_population.csv', 
                         usecols=['Code zone','Année','Valeur'])

# typage de noms de colonnes
population.rename(columns={'Code zone':'code_pays',
                           'Année' : 'annee',
                           'Valeur': 'population'}, inplace=True)

p3.china_duplicates(population)

# Convertir milliers d'habitants en habitant
population['population'] *= 1000

population = p3.dataframe_merge_pays(population)

# to order columns
population = population[['pays','code_pays','annee','population']]

# to export dataframe
population.to_csv('population.csv',index=False)

In [3]:
#CREATION DU CSV TABLE SOUS_NUTRITION

sous_nutrition = pd.read_csv('./DATA/fr_sousalimentation.csv', 
                             usecols=['Code zone','Code année','Valeur'])

sous_nutrition.rename(columns={'Code zone' :'code_pays',
                               'Code année':'annee',
                               'Valeur'    :'nb_personnes'},inplace=True)

p3.china_duplicates(sous_nutrition)

sous_nutrition = p3.dataframe_merge_pays(sous_nutrition)

# supprimer les lignes '<0.1'
sous_nutrition = sous_nutrition[sous_nutrition['nb_personnes']!='<0.1']

#suppression des lignes à 0 ou nulle
sous_nutrition.dropna(inplace=True)

#Formatage de la colonne annee de '2012XXXX' à '2013'
sous_nutrition['annee'] = (sous_nutrition['annee'] // 10000)+1

#convertir colonne nb_personnes de texte en nombre
sous_nutrition['nb_personnes'] = pd.to_numeric(sous_nutrition['nb_personnes'])

# to select annee = 2013 only
sous_nutrition = sous_nutrition[sous_nutrition['annee'] == 2013]

#convertir million de personnes en personne
sous_nutrition['nb_personnes'] *= 10**6

# type columns
sous_nutrition['code_pays'   ]=sous_nutrition['code_pays'   ].astype('int64')
sous_nutrition['nb_personnes']=sous_nutrition['nb_personnes'].astype('int64')
sous_nutrition['annee'       ]=sous_nutrition['annee'       ].astype('int64')

# to order columns
sous_nutrition = sous_nutrition[['pays','code_pays','annee','nb_personnes']]

sous_nutrition.to_csv('sous_nutrition.csv',index = False)

In [4]:
# TO CREATE : CORE DATAFRAME

# to load useful columns from 2 files with same structure
columns = ['Code zone', 'Code Élément', 'Code Produit', 'Année', 'Valeur', 'Symbole']

core_dataframe = pd.read_csv('./DATA/fr_vegetaux.csv',  usecols = columns)
anim_dataframe = pd.read_csv('./DATA/fr_animaux.csv' ,  usecols = columns)

# to create column for product's origin.
core_dataframe['origin'] = 'végétal'
anim_dataframe['origin'] = 'animal'

# to drop food balance sheet's duplicates : A for Aggregates
core_dataframe = core_dataframe[core_dataframe['Symbole']!='A']
anim_dataframe = anim_dataframe[anim_dataframe['Symbole']!='A']

# to merge 2 dataframes and delete useless variables
core_dataframe = core_dataframe.merge(anim_dataframe, how = 'outer')
del anim_dataframe, columns
del core_dataframe['Symbole']

core_dataframe.rename(columns={'Code zone':'code_pays', 
                               'Code Élément':'code_element', 
                               'Code Produit':'code_produit',
                               'Année':'annee',
                               'Valeur':'valeur'
                              },
                  
                      inplace=True
                     )


dictionnaire_code_element = {664 : 'dispo_alim_kcal_p_j',
                             674 : 'dispo_prot',
                             684 : 'dispo_mat_gr',
                             5301: 'dispo_int',
                             5521: 'alim_ani',
                             5527: 'semences',
                             5123: 'pertes',
                             5131: 'transfo',
                             5142: 'nourriture',
                             5154: 'autres_utilisations',
                             5511: 'productions',
                             5611: 'importations',
                             5911: 'exportations',
                             5072: 'variations'
                            }

# to delete china's duplicates
core_dataframe = p3.china_duplicates(core_dataframe)

# to delete code_element 645 : 'Disponibilité alimentaire en quantité'
core_dataframe = core_dataframe[core_dataframe['code_element'] != 645]

# to transpose code_element
core_dataframe = pd.pivot_table(core_dataframe,
                       index=['annee','code_pays','code_produit','origin'],
                       columns=['code_element'],
                       values=['valeur'],
                       fill_value='')

# drop 1 level of column names
core_dataframe.columns = core_dataframe.columns.droplevel(0)

#supprimer 'code_element' comme nom des colonnes
core_dataframe.columns = list(core_dataframe.columns)

# déplacer code_pays et produit dans le dataframe
core_dataframe.reset_index(inplace=True)

# to rename columns
core_dataframe.rename(columns = dictionnaire_code_element, inplace = True)

# to replace Nan by 0
core_dataframe.replace('',0,inplace=True)


# to merge core_dataframe with name_pays and name_produit
core_dataframe = p3.dataframe_merge_pays   (core_dataframe)
core_dataframe = p3.dataframe_merge_produit(core_dataframe)

# to create column 'dispo_alim_tonnes'
core_dataframe['dispo_alim_tonnes'] = core_dataframe['nourriture']

# to change unity : MT to T
core_dataframe['dispo_alim_tonnes'] *= 1000


# check first part equation integrity
p3.match_dispo_int(core_dataframe,'balance_com')

# check second part equation integrity
p3.match_dispo_int(core_dataframe, 'equilibre_prod')

# fix 2013 Japon Avoine
p3.fix_2013_japan_avoine(core_dataframe)

# set dispo_alim_kcal_p_j ,dispo_prot and dispo_mat_gr to 0 if nourriture = 0
p3.match_availabilities_nourriture_0(core_dataframe)

# delete useless rows
index_0 = core_dataframe[ (core_dataframe[dictionnaire_code_element[ 664]]==0)
                         &(core_dataframe[dictionnaire_code_element[ 684]]==0)
                         &(core_dataframe[dictionnaire_code_element[5301]]==0)
                         &(core_dataframe[dictionnaire_code_element[5521]]==0)
                         &(core_dataframe[dictionnaire_code_element[5527]]==0)
                         &(core_dataframe[dictionnaire_code_element[5123]]==0)
                         &(core_dataframe[dictionnaire_code_element[5131]]==0)
                         &(core_dataframe[dictionnaire_code_element[5142]]==0)
                         &(core_dataframe[dictionnaire_code_element[5154]]==0)
                         &(core_dataframe[dictionnaire_code_element[5511]]==0)
                         &(core_dataframe[dictionnaire_code_element[5611]]==0)
                         &(core_dataframe[dictionnaire_code_element[5911]]==0)
                         &(core_dataframe[dictionnaire_code_element[5072]]==0)
                         &(core_dataframe[dictionnaire_code_element[ 674]]==0)
                        ].index

core_dataframe.drop(index = index_0,inplace=True)


# to type columns

core_dataframe['annee']               = core_dataframe['annee'              ].astype(  'int64')
core_dataframe['code_pays']           = core_dataframe['code_pays'          ].astype(  'int64')
core_dataframe['code_produit']        = core_dataframe['code_produit'       ].astype(  'int64')

core_dataframe['dispo_alim_tonnes']   = core_dataframe['dispo_alim_tonnes'  ].astype(  'int64')
core_dataframe['dispo_alim_kcal_p_j'] = core_dataframe['dispo_alim_kcal_p_j'].astype('float64')
core_dataframe['dispo_prot']          = core_dataframe['dispo_prot'         ].astype('float64')
core_dataframe['dispo_mat_gr']        = core_dataframe['dispo_mat_gr'       ].astype('float64')


core_dataframe['dispo_int']           = core_dataframe['dispo_int'          ].astype(  'int64')
core_dataframe['alim_ani']            = core_dataframe['alim_ani'           ].astype(  'int64')
core_dataframe['semences']            = core_dataframe['semences'           ].astype(  'int64')
core_dataframe['pertes']              = core_dataframe['pertes'             ].astype(  'int64')
core_dataframe['transfo']             = core_dataframe['transfo'            ].astype(  'int64')
core_dataframe['nourriture']          = core_dataframe['nourriture'         ].astype(  'int64')
core_dataframe['autres_utilisations'] = core_dataframe['autres_utilisations'].astype(  'int64')

core_dataframe['productions']         = core_dataframe['productions'        ].astype(  'int64')
core_dataframe['importations']        = core_dataframe['importations'       ].astype(  'int64')
core_dataframe['exportations']        = core_dataframe['exportations'       ].astype(  'int64')
core_dataframe['variations']          = core_dataframe['variations'         ].astype(  'int64')

In [5]:
# CREATE TABLE dispo_alim

columns_list = ['pays',
                'code_pays',
                'annee',
                'produit',
                'code_produit',
                'origin',
                'dispo_alim_tonnes',
                'dispo_alim_kcal_p_j',
                'dispo_prot',
                'dispo_mat_gr'
               ]

dispo_alim = p3.dataframe_table('dispo_alim',columns_list,core_dataframe)

# CREATE TABLE equilibre_prod

columns_list = ['pays',
                'code_pays',
                'annee',
                'produit',
                'code_produit',
                'dispo_int',
                'alim_ani',
                'semences',
                'pertes',
                'transfo',
                'nourriture',
                'autres_utilisations'
               ]

equilibre_prod = p3.dataframe_table('equilibre_prod',columns_list,core_dataframe)

# CREATE TABLE balance_com

columns_list = ['pays',
                'code_pays',
                'annee',
                'produit',
                'code_produit',
                'productions',
                'importations',
                'exportations',
                'variations'
               ]

balance_com = p3.dataframe_table('balance_com',columns_list,core_dataframe)